# Introduction

In [1]:
import stormpy
import stormpy.core

Load the *agent stochastic maze* MDP model :

In [2]:
path = '../../examples/simple_mdp.prism'
prism_program = stormpy.parse_prism_program(path)

In [3]:
with open(path, 'r') as prism_file:
    print(prism_file.read())

mdp

module classic

s: [0..2] init 0;

[beta] s=0 -> 0.5 : (s'=1) + 0.5 : (s'=2);
[gamma] s=1 -> 1 : (s'=0);
[alpha] s=2 -> 1 : (s'=2);
[gamma] s=2 -> 1 : (s'=0);

endmodule

label "T" = s=1;

rewards "weights"
  [alpha] true : 5;
  [beta] true : 3;
  [gamma] true : 2;
endrewards



In [5]:
model = stormpy.build_model(prism_program)
print("Model Type: {}".format(model.model_type))
print("Number of states: {}".format(model.nr_states))
print("Number of transitions: {}".format(model.nr_transitions))

Model Type: ModelType.MDP
Number of states: 3
Number of transitions: 5


In [6]:
print(model.transition_matrix)

		0	1	2	
	---- group 0/2 ---- 
0	(	0	0.5	0.5		)	0
	---- group 1/2 ---- 
1	(	1	0	0		)	1
	---- group 2/2 ---- 
2	(	1	0	0		)	2
3	(	0	0	1		)	3
		0	1	2	



Solve the stochastic shortest path expectation problem :

In [7]:
# Prism-PCTL formula
formula = 'Rmin=? [F "T"]'
properties = stormpy.parse_properties_for_prism_program(formula, prism_program)

In [8]:
model = stormpy.build_model(prism_program, properties)
print(model.labeling)
#print("Labels in the model: {}".format(sorted(model.labeling.get_labels())))

3 labels
   * init -> 1 item(s)
   * deadlock -> 0 item(s)
   * T -> 1 item(s)



In [9]:
result = stormpy.model_checking(model, properties[0])

In [10]:
assert result.result_for_all_states
for s, x in enumerate(result.get_values()):
    print("v[{}]={}".format(s, x))
#print(result.get_values())

v[0]=8.0
v[1]=0.0
v[2]=10.0


# Investigating the model

In [11]:
for state in model.states:
    for action in state.actions:
        descr = "[a{}:{}] s={} -> ".format(state, action, state)
        for transition in action.transitions:
            #print("From state {}, choose the action {} and go to state {} with probability {}".format(state, action, transition.column, transition.value()))
            descr += "{} : (s'={}) + ".format(transition.value(), transition.column)
        print(descr[:-2])

[a0:0] s=0 -> 0.5 : (s'=1) + 0.5 : (s'=2) 
[a1:0] s=1 -> 1.0 : (s'=0) 
[a2:0] s=2 -> 1.0 : (s'=0) 
[a2:1] s=2 -> 1.0 : (s'=2) 


In [12]:
type(model)

stormpy.storage.storage.SparseMdp

# Rewards

In [13]:
reward_model_name = list(model.reward_models.keys())[0]
reward_model_name

'weights'

In [14]:
assert(model.reward_models[reward_model_name].has_state_action_rewards)
assert not model.reward_models[reward_model_name].has_state_rewards
assert not model.reward_models[reward_model_name].has_transition_rewards

model.reward_models[reward_model_name].state_action_rewards

[3.0, 2.0, 2.0, 5.0]

# Model exportation

In [19]:
stormpy.export_to_drn(model, 'model_drn')

In [20]:
with open('model_drn', 'r') as prism_file:
    print(prism_file.read())

// Exported by storm
// Original model type: MDP
@type: MDP
@parameters

@reward_models
weights 
@nr_states
3
@model
state 0 [0] init
	action 0 [3]
		1 : 0.5
		2 : 0.5
state 1 [0] T
	action 0 [2]
		0 : 1
state 2 [0]
	action 0 [2]
		0 : 1
	action 1 [5]
		2 : 1

